# Player stats by game fetching from Sportradar and Supabase update

In [1]:
import os
import sys
from django.conf import settings
import pandas as pd
from datetime import datetime
import time
import requests

In [2]:
# Set up the Django environment
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "hooperstats.settings")
import django
django.setup()

In [3]:
# Import utility functions
from api.sportradar_utils import get_sportradar_data
from api.supabase_utils import get_supabase_client

In [4]:
# Get Supabase client
supabase = get_supabase_client()

In [5]:
# Fetch Phoenix Suns team ID
suns_team = supabase.table('teams').select('*').eq('name', 'Phoenix Suns').execute()
suns_id = suns_team.data[0]['id']

# Fetch Booker and Durant player IDs
players = supabase.table('players').select('*').in_('name', ['Devin Booker', 'Kevin Durant']).execute()
player_ids = {player['name']: player['id'] for player in players.data}

print(f"Suns ID: {suns_id}")
print(f"Player IDs: {player_ids}")

Suns ID: 583ecfa8-fb46-11e1-82cb-f4ce4684ea4c
Player IDs: {'Devin Booker': '31baa84f-c759-4f92-8e1f-a92305ade3d6', 'Kevin Durant': '53f2fa48-e61b-49fb-843d-8a3e872257eb'}


In [6]:
# Fetch all Suns games
suns_games = supabase.table('games').select('*').or_(f"home_team_id.eq.{suns_id},away_team_id.eq.{suns_id}").execute()
games_df = pd.DataFrame(suns_games.data)
print(f"Total Suns games: {len(games_df)}")
games_df.head()

Total Suns games: 82


,id,date,season,game_number,home_team_id,away_team_id,winner_team_id,home_team_name,away_team_name,winner_team_name,home_points,away_points
0,b06ae4da-247e-4696-973a-7f1cbd813fec,2023-10-25,2023,1,583ec825-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Golden State Warriors,Phoenix Suns,Phoenix Suns,104,108
1,6b977a9a-8eec-44c3-85ca-ec8787d7dad0,2023-10-27,2023,2,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecae2-fb46-11e1-82cb-f4ce4684ea4c,Los Angeles Lakers,Phoenix Suns,Los Angeles Lakers,100,95
2,9393ca30-2192-4341-9770-20c95ff426b5,2023-10-29,2023,3,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ece50-fb46-11e1-82cb-f4ce4684ea4c,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,Utah Jazz,Phoenix Suns,126,104
3,3d03bd2b-a31e-4b6d-8ffc-adae63a0e000,2023-11-01,2023,4,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,San Antonio Spurs,114,115
4,f9f4b8f6-42fd-448a-a757-c3224c5201a1,2023-11-03,2023,5,583ecfa8-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,583ecd4f-fb46-11e1-82cb-f4ce4684ea4c,Phoenix Suns,San Antonio Spurs,San Antonio Spurs,121,132


In [7]:
import logging

def fetch_game_summary(game_id):
    try:
        summary_data = get_sportradar_data(f'games/{game_id}/summary.json')
    except Exception as e:
        logging.error(f"Failed to fetch game summary for game {game_id}: {str(e)}")
        return []

    player_stats = []
    for team in ['home', 'away']:
        if team not in summary_data:
            logging.warning(f"Team data '{team}' not found in game summary for game {game_id}")
            continue

        for player in summary_data[team].get('players', []):
            try:
                if player['id'] in player_ids.values():
                    stats = {
                        'player_id': player['id'],
                        'player_name': next((name for name, pid in player_ids.items() if pid == player['id']), None),
                        'game_id': game_id,
                        'three_points_made': player['statistics'].get('three_points_made', 0),
                        'three_points_att': player['statistics'].get('three_points_att', 0),
                        'three_points_pct': player['statistics'].get('three_points_pct', 0)
                    }
                    if stats['player_name'] is None:
                        logging.warning(f"Player name not found for ID {player['id']} in game {game_id}")
                    player_stats.append(stats)
            except KeyError as e:
                logging.error(f"Missing key in player data for game {game_id}: {str(e)}")
            except Exception as e:
                logging.error(f"Error processing player in game {game_id}: {str(e)}")

    return player_stats

# Test the function with one game
try:
    test_game_id = games_df.iloc[0]['id']
    result = fetch_game_summary(test_game_id)
    print(f"Fetched {len(result)} player stats for game {test_game_id}")
    print(result)
except Exception as e:
    print(f"An error occurred while testing: {str(e)}")

Fetched 2 player stats for game b06ae4da-247e-4696-973a-7f1cbd813fec
[{'player_id': '53f2fa48-e61b-49fb-843d-8a3e872257eb', 'player_name': 'Kevin Durant', 'game_id': 'b06ae4da-247e-4696-973a-7f1cbd813fec', 'three_points_made': 1, 'three_points_att': 2, 'three_points_pct': 50.0}, {'player_id': '31baa84f-c759-4f92-8e1f-a92305ade3d6', 'player_name': 'Devin Booker', 'game_id': 'b06ae4da-247e-4696-973a-7f1cbd813fec', 'three_points_made': 3, 'three_points_att': 8, 'three_points_pct': 37.5}]


In [8]:
def process_all_games(games_df, delay=1):
    all_stats = []
    for index, game in games_df.iterrows():
        print(f"Processing game {index + 1}/{len(games_df)}")
        game_stats = fetch_game_summary(game['id'])
        for stats in game_stats:
            stats['team_game_number'] = game['game_number']
            stats['date'] = game['date']
            stats['team_won'] = (game['winner_team_id'] == suns_id)
            
            # Ensure all required fields are present
            stats['three_points_made'] = stats.get('three_points_made', 0)
            stats['three_points_att'] = stats.get('three_points_att', 0)
            stats['three_points_pct'] = stats.get('three_points_pct', 0)  # Note the change from three_point_percentage to three_points_pct
            
        all_stats.extend(game_stats)
        time.sleep(delay)  # Add delay to avoid API rate limits
    return pd.DataFrame(all_stats)

player_stats_df = process_all_games(games_df)
print(player_stats_df.head())
print(f"Total player stats records: {len(player_stats_df)}")

Processing game 1/82
Processing game 2/82
Processing game 3/82
Processing game 4/82
Processing game 5/82
Processing game 6/82
Processing game 7/82
Processing game 8/82
Processing game 9/82
Processing game 10/82
Processing game 11/82
Processing game 12/82
Processing game 13/82
Processing game 14/82
Processing game 15/82
Processing game 16/82
Processing game 17/82
Processing game 18/82
Processing game 19/82
Processing game 20/82
Processing game 21/82
Processing game 22/82
Processing game 23/82
Processing game 24/82
Processing game 25/82
Processing game 26/82
Processing game 27/82
Processing game 28/82
Processing game 29/82
Processing game 30/82
Processing game 31/82
Processing game 32/82
Processing game 33/82
Processing game 34/82
Processing game 35/82
Processing game 36/82
Processing game 37/82
Processing game 38/82
Processing game 39/82
Processing game 40/82
Processing game 41/82
Processing game 42/82
Processing game 43/82
Processing game 44/82
Processing game 45/82
Processing game 46/

ERROR:root:Failed to fetch game summary for game f9998a42-9c4e-41b4-bd98-0c30012b331c: 429 Client Error: Too Many Requests for url: https://api.sportradar.com/nba/trial/v8/en/games/f9998a42-9c4e-41b4-bd98-0c30012b331c/summary.json?api_key=nI7G8NkRAVacD3DK1edkc7NTRiJAh5zv40Gdb6aQ


Processing game 81/82
Processing game 82/82
                              player_id   player_name  \
0  53f2fa48-e61b-49fb-843d-8a3e872257eb  Kevin Durant   
1  31baa84f-c759-4f92-8e1f-a92305ade3d6  Devin Booker   
2  53f2fa48-e61b-49fb-843d-8a3e872257eb  Kevin Durant   
3  31baa84f-c759-4f92-8e1f-a92305ade3d6  Devin Booker   
4  53f2fa48-e61b-49fb-843d-8a3e872257eb  Kevin Durant   

                                game_id  three_points_made  three_points_att  \
0  b06ae4da-247e-4696-973a-7f1cbd813fec                  1                 2   
1  b06ae4da-247e-4696-973a-7f1cbd813fec                  3                 8   
2  6b977a9a-8eec-44c3-85ca-ec8787d7dad0                  1                 5   
3  6b977a9a-8eec-44c3-85ca-ec8787d7dad0                  0                 0   
4  9393ca30-2192-4341-9770-20c95ff426b5                  1                 4   

   three_points_pct  team_game_number        date  team_won  
0              50.0                 1  2023-10-25      True  
1       

In [9]:
import uuid

# Ensure all required columns are present and in the correct order
required_columns = [
    'id', 'player_id', 'player_name', 'game_id', 'team_game_number', 
    'date', 'team_won', 'three_points_made', 'three_points_att', 'three_points_pct'
]

# Add missing columns if any
for column in required_columns:
    if column not in player_stats_df.columns:
        if column == 'id':
            player_stats_df['id'] = [str(uuid.uuid4()) for _ in range(len(player_stats_df))]
        else:
            player_stats_df[column] = None  # or some default value

# Reorder columns
player_stats_df = player_stats_df[required_columns]

# Convert data types to match the table definition
player_stats_df['team_game_number'] = player_stats_df['team_game_number'].astype(int)
player_stats_df['date'] = pd.to_datetime(player_stats_df['date']).dt.date
player_stats_df['team_won'] = player_stats_df['team_won'].astype(bool)
player_stats_df['three_points_made'] = player_stats_df['three_points_made'].astype(int)
player_stats_df['three_points_att'] = player_stats_df['three_points_att'].astype(int)
player_stats_df['three_points_pct'] = player_stats_df['three_points_pct'].astype(float)

# Display the first few rows to verify
print(player_stats_df.head())
print(f"Total records to insert: {len(player_stats_df)}")

                                     id                             player_id  \
0  93c60a16-7303-4d54-a4bc-347524ba1a2a  53f2fa48-e61b-49fb-843d-8a3e872257eb   
1  a7885083-5c83-4e61-9a2c-6f679adf4641  31baa84f-c759-4f92-8e1f-a92305ade3d6   
2  0bc3e7dc-f44f-4063-a425-c1b58914e8af  53f2fa48-e61b-49fb-843d-8a3e872257eb   
3  e60390d3-d0f0-4d58-8c8c-17cae16d2fc8  31baa84f-c759-4f92-8e1f-a92305ade3d6   
4  03802a05-e49e-4a33-ad36-52c9469e6477  53f2fa48-e61b-49fb-843d-8a3e872257eb   

    player_name                               game_id  team_game_number  \
0  Kevin Durant  b06ae4da-247e-4696-973a-7f1cbd813fec                 1   
1  Devin Booker  b06ae4da-247e-4696-973a-7f1cbd813fec                 1   
2  Kevin Durant  6b977a9a-8eec-44c3-85ca-ec8787d7dad0                 2   
3  Devin Booker  6b977a9a-8eec-44c3-85ca-ec8787d7dad0                 2   
4  Kevin Durant  9393ca30-2192-4341-9770-20c95ff426b5                 3   

         date  team_won  three_points_made  three_points_att  

In [11]:
# Convert date to string format
player_stats_df['date'] = player_stats_df['date'].astype(str)

# Convert DataFrame to list of dictionaries
player_stats_records = player_stats_df.to_dict('records')

# Insert data into Supabase
result = supabase.table('player_stats_by_game').insert(player_stats_records).execute()

print(f"Inserted {len(result.data)} records into player_stats_by_game table")

Inserted 162 records into player_stats_by_game table
